In [31]:
import pandas as pd, numpy as np

In [165]:
df = pd.read_csv('../out/cluster_featurize/cluster_features.csv', index_col=0)
print(df.shape)

(7334, 42)


In [166]:
df.columns

Index(['Feature - Hold', 'Feature - Hold taps', 'Feature - Splits',
       'Feature - Triple', 'Feature - Quad', 'Feature - Jump',
       'Feature - Bracket', 'Feature - Double step', 'Feature - Footswitch',
       'Feature - Jack', 'Feature - Bracket footswitch',
       'Feature - Hold tap single foot', 'Feature - Hold footslide',
       'Feature - Hold footswitch', 'Feature - Staggered hit',
       'Feature - Hands', 'Feature - Drill', 'Feature - Run',
       'Feature - Hold run', 'Feature - Bracket drill',
       'Feature - Irregular rhythm', 'Feature - Rhythm change',
       'Feature - Bracket jump run', 'Feature - Side3 singles',
       'Feature - Mid4 doubles', 'Feature - Mid6 doubles',
       'Feature - Run with brackets', 'Feature - Jump run',
       'Feature - Stairs, doubles', 'Feature - Stairs, singles',
       'Feature - Broken stairs, doubles', 'Feature - Spin',
       'Feature - Twist solo diagonal', 'Feature - Twist angle - none',
       'Feature - Twist angle - 90', 'Fe

### test cluster

In [169]:
df_subset = df.drop(columns = [
    'Feature - Hold',
    'Feature - Hold taps',
    'Feature - Hold run',
    'Feature - Irregular rhythm',
    'Feature - Rhythm change',
])

ft_cols = [col for col in df_subset.columns if 'Feature - ' in col]
print(ft_cols)

['Feature - Splits', 'Feature - Triple', 'Feature - Quad', 'Feature - Jump', 'Feature - Bracket', 'Feature - Double step', 'Feature - Footswitch', 'Feature - Jack', 'Feature - Bracket footswitch', 'Feature - Hold tap single foot', 'Feature - Hold footslide', 'Feature - Hold footswitch', 'Feature - Staggered hit', 'Feature - Hands', 'Feature - Drill', 'Feature - Run', 'Feature - Bracket drill', 'Feature - Bracket jump run', 'Feature - Side3 singles', 'Feature - Mid4 doubles', 'Feature - Mid6 doubles', 'Feature - Run with brackets', 'Feature - Jump run', 'Feature - Stairs, doubles', 'Feature - Stairs, singles', 'Feature - Broken stairs, doubles', 'Feature - Spin', 'Feature - Twist solo diagonal', 'Feature - Twist angle - none', 'Feature - Twist angle - 90', 'Feature - Twist angle - close diagonal', 'Feature - Twist angle - far diagonal', 'Feature - Twist angle - 180']


In [170]:
nm = 'Super Fantasy - SHK S16 arcade'

def get_features(df, nm):
    ft_cols = [col for col in df.columns if 'Feature - ' in col]
    return np.array(df.loc[df['Name (unique)'] == nm, ft_cols])

In [171]:
def get_feature_mat(df):
    ft_df = df.drop(columns=[c for c in df.columns if 'Feature - ' not in c])
    return np.array(ft_df)

# ft_mat = get_feature_mat(df)
ft_mat = get_feature_mat(df_subset)
POWER = 1
ft_mat = ft_mat**POWER

In [173]:
nms = list(df['Name (unique)'])
levels = list(df['Level'])
idx = nms.index(nm)

high_pct_jdxs = [i for i,v in enumerate(ft_mat[idx]) if v >= 0.65**POWER]
print(high_pct_jdxs)
print([ft_cols[j] for j in high_pct_jdxs])

[15, 28]
['Feature - Run', 'Feature - Twist angle - none']


In [174]:
from scipy.spatial import KDTree

HIGH_PCT_THRESHOLD = 0.65**POWER

def get_neighbors(df, nm):
    nms = list(df['Name (unique)'])
    levels = list(df['Level'])
    idx = nms.index(nm)

    high_pct_jdxs = [i for i,v in enumerate(ft_mat[idx]) if v >= HIGH_PCT_THRESHOLD]
#     print([ft_cols[j] for j in high_pct_jdxs])
#     high_pct_jdxs += [i for i,v in enumerate(ft_mat[idx]) if v == 0]
#     high_pct_jdxs = [ft_cols.index('Feature - Run'), ft_cols.index('Feature - Twist angle - none')]
    print([ft_cols[j] for j in high_pct_jdxs])
    high_pct_ft_mat = ft_mat[:, high_pct_jdxs]
    
    kdt = KDTree(high_pct_ft_mat)
#     q = high_pct_ft_mat[idx]
    q = [1] * len(high_pct_jdxs)
    dists, ne_idxs = kdt.query(q, k=1000)

    num_printed = 0
    from collections import defaultdict
    top3s = defaultdict(list)
    lower_level_range = 3
    upper_level_range = 2
    in_range = lambda neighbor_level: levels[idx] - 3 <= neighbor_level <= levels[idx] + 2
    for dist, ne_idx in zip(dists[1:], ne_idxs[1:]):
        target_level = levels[ne_idx]
        if in_range(target_level):
#             print(f'{dist:.2f}', nms[ne_idx])
            if len(top3s[target_level]) < 3:
                top3s[target_level].append(nms[ne_idx])
            num_printed += 1
        if num_printed > 50:
            break
    
    for k in sorted(top3s):
        print(k, top3s[k])
    return


def get_dist(df, nm1, nm2):
    nms = list(df['Name (unique)'])
    levels = list(df['Level'])
    idx = nms.index(nm)

    high_pct_jdxs = [i for i,v in enumerate(ft_mat[idx]) if v >= HIGH_PCT_THRESHOLD]
    high_pct_jdxs += [i for i,v in enumerate(ft_mat[idx]) if v == 0]
    high_pct_ft_mat = ft_mat[:, high_pct_jdxs]
    
    dist = np.linalg.norm(high_pct_ft_mat[idx] - high_pct_ft_mat[nms.index(nm2)])
    return dist


# get_neighbors(df_subset, 'Super Fantasy - SHK S16 arcade')
get_neighbors(df, 'Super Fantasy - SHK S16 arcade')

['Feature - Run', 'Feature - Twist angle - none']
13 ['Love is a Danger Zone - Banya S13 shortcut', 'God Mode 2.0 feat. Skizzo - Nato S13 arcade', 'Mexi Mexi - Mozquito S13 arcade']
14 ['Switronic - SHORT CUT - - SHK S14 shortcut', 'HEART RABBIT COASTER - nato S14 arcade', 'Accelerator - Nitro S14 arcade']
15 ['X Treme - BanYa S15 arcade', 'Vook - Banya S15 arcade', 'V.I.P. - Bada D15 arcade']
16 ["I'll Give You All My Love - Wax S16 arcade", 'PaPa Gonzales - BanYa S16 arcade v2', 'PaPa Gonzales - BanYa S16 arcade v1']
17 ['You Fall in Me - No Brain S17 arcade', 'msgoon RMX pt.3 - msgoon S17 remix', 'Conflict - Siromaru + Cranky S17 arcade']
18 ['Poseidon - SHORT CUT - - Quree S18 shortcut', 'Heavy Rotation - AKB48 (Cover) S18 shortcut', 'God Mode 2.0 feat. Skizzo - Nato S18 arcade']


### test queries

In [175]:
get_neighbors(df, 'Trashy Innocence - Last Note. D15 arcade')

['Feature - Jump', 'Feature - Mid4 doubles', 'Feature - Mid6 doubles', 'Feature - Twist angle - 90', 'Feature - Twist angle - 180']
12 ['Rock Robotic -OSX Mix- - Oscillator X HD12 arcade', 'Switronic - SHORT CUT - - SHK D12 shortcut', 'Red Swan - Yahpp D12 arcade']
13 ["Bullfighting's Song - Banya Production HD13 arcade", 'Ba Be Loo Be Ra - Kristeen D13 arcade', 'Turn Around - Mina D13 arcade']
14 ["One Love - S'Max D14 arcade hidden", 'Venus - Shinhwa D14 arcade', 'YOU AND I - Dreamcatcher D14 arcade']
15 ['Trashy Innocence - Last Note. D15 arcade', 'Jackpot - BLOCK B D15 arcade', 'NoNoNo - Apink DP15 arcade']
16 ['Removable Disk0 - Doin D16 arcade', 'Houkago Stride - Last Note. D16 arcade', 'Black Cat - Turbo D16 arcade']
17 ['Crossing Delta - t+pazolite+Ginkiha+Hommarju D17 arcade', 'NUMBER NINE - T-ARA D17 arcade v1', 'NUMBER NINE - T-ARA D17 arcade v2']


In [176]:
# get_neighbors(df_subset, '8 6 - DASU S20 arcade')
get_neighbors(df, '8 6 - DASU S20 arcade')

['Feature - Bracket', 'Feature - Double step', 'Feature - Twist angle - none']
17 ['Imagination - SHK S17 arcade', 'Emperor - BanYa D17 arcade', 'Hell Flame - Diclonius Kid S17 arcade']
18 ['Skeptic - Doin S18 arcade', 'Dance Vibrations - Elpis D18 arcade', 'Red Swan - Yahpp S18 arcade']
19 ['Break Out - Lunatic Sounds S19 arcade', 'Bring Back The Beat - LunaticSounds S19 arcade', 'Mission Possible - BanYa S19 arcade']
20 ['Mad5cience - Paul Bazooka S20 arcade', 'Kimchi Fingers - Garlic Squad S20 arcade', 'CHICKEN WING - Banya S20 shortcut']
21 ['Headless Chicken - r300k S21 arcade', 'Your Mind - Roy Mikelate S21 arcade', 'Pop Sequence - Hommarju S21 arcade']
22 ['Dream To Nightmare - Nightmare S22 arcade hidden', 'DESTRUCIMATE - KURORAK S22 arcade', 'Phalanx "RS2018 Edit" - Cranky S22 arcade']


In [177]:
get_neighbors(df, 'Rock the house - Matduke D22 arcade')

['Feature - Bracket', 'Feature - Bracket footswitch', 'Feature - Hold tap single foot', 'Feature - Staggered hit', 'Feature - Drill', 'Feature - Bracket drill', 'Feature - Mid4 doubles', 'Feature - Mid6 doubles', 'Feature - Twist angle - none']
19 ['ESCAPE - D_AAN D19 arcade v1', 'ESCAPE - D_AAN D19 arcade v2', 'ESCAPE - D_AAN D19 arcade v3']
20 ['Enhanced Reality - Matduke D20 arcade', 'Red Swan - Yahpp D20 arcade', 'Annihilator Method - DM Ashura D20 arcade']
21 ['Feel My Happiness - 3R2 D21 arcade', 'Dawn of the Apocalypse - Vassline D21 arcade', 'Ignis Fatuus(DM Ashura Mix) - DM Ashura D21 shortcut']
22 ['Indestructible - Matduke D22 arcade', 'What Happened - Throwdown HD22 arcade', 'Just Hold on (To All Fighters) - void D22 arcade']
23 ['Can-can ~Orpheus in The Party Mix~ - Sr. Lan Belmont D23 arcade', 'Windmill - Yak Won D23 arcade', 'Bad End Night - HitoshizukuP x yama D23 arcade']
24 ['Vulcan - Memme D24 remix v2', 'Banya-P Guitar Remix - Banya Production D24 remix', 'Iolite Sk

### study features

In [104]:
get_features(df_subset, 'Super Fantasy - SHK S16 arcade')

array([[0.        , 0.        , 0.        , 0.0272259 , 0.        ,
        0.00883002, 0.        , 0.18543046, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.68727005, 0.        , 0.5857248 , 0.05739514, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.71743929, 0.        , 0.        , 0.07505519,
        0.89551141, 0.16850625, 0.17218543, 0.        , 0.        ]])

In [29]:
get_features(df_subset, 'X-Rave - DM Ashura S17 arcade')

array([[0.        , 0.        , 0.        , 0.2520436 , 0.5006812 ,
        0.35149864, 0.        , 0.10490463, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.73092643, 0.        , 0.86307902, 0.49318801, 0.        ,
        0.51089918, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.54700272, 0.        , 0.        , 0.28474114,
        0.79427793, 0.26702997, 0.29972752, 0.        , 0.        ]])

In [30]:
get_features(df_subset, 'Conflict - Siromaru + Cranky S17 arcade')

array([[0.        , 0.        , 0.        , 0.19891008, 0.        ,
        0.52656676, 0.        , 0.69073569, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.88623978,
        0.8726158 , 0.        , 0.48024523, 0.08038147, 0.        ,
        0.90735695, 0.        , 0.        , 0.        , 0.67166213,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.98841962, 0.01362398, 0.01907357, 0.55722071, 0.        ]])

In [33]:
np.linalg.norm(
    get_features(df_subset, 'Super Fantasy - SHK S16 arcade') -
    get_features(df_subset, 'Conflict - Siromaru + Cranky S17 arcade')
)

1.8829833110672938

In [168]:
comp = [
    'Super Fantasy - SHK S16 arcade', 
    'Conflict - Siromaru + Cranky S17 arcade',
    'Hi-Bi - BanYa S16 arcade',
]
df_subset[df_subset['Name (unique)'].isin(comp)].T

,3588,3676,3881
Feature - Splits,0,0,0
Feature - Triple,0,0,0
Feature - Quad,0,0,0
Feature - Jump,0,0,0
Feature - Bracket,0,0,0
Feature - Double step,0,0,0.526567
Feature - Footswitch,0,0,0
Feature - Jack,0,0,0
Feature - Bracket footswitch,0,0,0
Feature - Hold tap single foot,0,0,0


In [87]:
get_dist(
    df_subset, 
    '8 6 - DASU S20 arcade',
    'Nakakapagpabagabag - Dasu feat. Kagamine Len S18 arcade',
)

0.7294292988043192

In [88]:
get_dist(
    df_subset, 
    '8 6 - DASU S20 arcade',
    'GLORIA - Croire S23 arcade',
)

0.17263193944135724

In [77]:
comp = [
    '8 6 - DASU S20 arcade',
    'Nakakapagpabagabag - Dasu feat. Kagamine Len S18 arcade',
]
df_subset[df_subset['Name (unique)'].isin(comp)].T

,3950,4369
Feature - Splits,0,0
Feature - Triple,0.996149,0.521706
Feature - Quad,0,0
Feature - Jump,0.921694,0.0693069
Feature - Bracket,0.990372,0.928408
Feature - Double step,0.842105,0.928408
Feature - Footswitch,0,0
Feature - Jack,0.292041,0
Feature - Bracket footswitch,0,0
Feature - Hold tap single foot,0.981386,0.965727
